In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="bartowski/Llama-3.2-1B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "hola ablame en español porfa"}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content='¡Hola, amigo mío,\nDe repito: ¡Hola, ablame en español, yo!\n\nQuería que hablemos de este tema,\nEl fuego rojo que te llama.\n\n"¡Ablama en español", esa es la respuesta,\nPuedes responder y responder, cada vez más.\n\n¿Cuál es tu respuesta?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [2]:
import os
import urllib.parse  # Para codificar la búsqueda en la URL
from openai import OpenAI

# Configuración del cliente API de OpenAI
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Función para ejecutar comandos del sistema y realizar búsquedas en YouTube
def ejecutar_comando(comando):
    comando = comando.lower()
    
    # Si el usuario quiere abrir solo YouTube
    if "abrir youtube" in comando and "buscar" not in comando:
        os.system("start https://www.youtube.com")  # En Windows
        # os.system("open https://www.youtube.com")  # En macOS
        # os.system("xdg-open https://www.youtube.com")  # En Linux
        return "Abriendo YouTube..."
    
    # Si el usuario quiere buscar algo en YouTube
    elif "buscar" in comando and "youtube" in comando:
        # Extraer el término de búsqueda del comando
        termino_busqueda = comando.split("buscar")[1].strip()
        query = urllib.parse.quote(termino_busqueda)  # Codificar la búsqueda para la URL
        url_busqueda = f"https://www.youtube.com/results?search_query={query}"
        
        # Ejecutar el comando del sistema para abrir el navegador con la búsqueda
        os.system(f"start {url_busqueda}")  # En Windows
        # os.system(f"open {url_busqueda}")  # En macOS
        # os.system(f"xdg-open {url_busqueda}")  # En Linux
        
        return f"Buscando '{termino_busqueda}' en YouTube..."
    
    else:
        return "Comando no reconocido."

# Bucle principal de la conversación
history = [
    {"role": "system", "content": "You are an assistant that can execute system commands like opening YouTube or searching on YouTube."},
]

while True:
    # Solicitud al modelo
    completion = client.chat.completions.create(
        model="bartowski/Llama-3.2-1B-Instruct-GGUF",
        messages=history,
        temperature=0.7,
        stream=True,
    )

    # Almacena la respuesta del modelo
    respuesta_modelo = ""
    for chunk in completion:
        if chunk.choices[0].delta.content:
            respuesta_modelo += chunk.choices[0].delta.content

    # Imprimir respuesta del modelo
    print(f"Asistente: {respuesta_modelo}")

    # Verificar si hay un comando a ejecutar
    respuesta_usuario = input("> ")
    history.append({"role": "user", "content": respuesta_usuario})

    # Verifica si el usuario quiere ejecutar un comando
    resultado_comando = ejecutar_comando(respuesta_usuario)
    print(f"{resultado_comando}")


Asistente:  What is the command to open a specific video on YouTube?


In [2]:
from openai import OpenAI

# Configurar el cliente para conectarse al servidor local
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Función para obtener el embedding de un texto
def get_embedding(text, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
    text = text.replace("\n", " ")  # Limpiar el texto
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Función para hacer que la IA responda en rima usando embeddings como contexto
def ask_in_rhyme(text):
    embedding = get_embedding(text)  # Obtener el embedding del texto ingresado
    
    # Hacer una solicitud a la IA para responder en rima
    completion = client.chat.completions.create(
        model="bartowski/Llama-3.2-1B-Instruct-GGUF",
        messages=[
            {"role": "system", "content": "Always answer in rhymes."},
            {"role": "user", "content": f"Here is some context based on the embedding: {embedding}. Now respond to this: {text}"}
        ],
        temperature=0.7,
    )
    
    # Imprimir la respuesta en rima
    return completion.choices[0].message['content']

# Bucle interactivo para que el usuario escriba y la IA responda
def interactive_chat():
    print("Chat interactivo con la IA (presiona Ctrl+C para salir)\n")
    while True:
        # Leer el input del usuario
        user_input = input("Tú: ")
        
        # Si el usuario no ingresa nada, terminamos el ciclo
        if not user_input:
            print("Terminando la conversación...")
            break
        
        # Obtener la respuesta de la IA
        response = ask_in_rhyme(user_input)
        
        # Mostrar la respuesta de la IA
        print(f"IA: {response}\n")

# Iniciar el chat interactivo
if __name__ == "__main__":
    interactive_chat()


Chat interactivo con la IA (presiona Ctrl+C para salir)

